# MLM 모델을 사용하여 문장의 부자연스러움 판단하기
# 원시와 차운시 비교하기

🚩 시인이 자유롭게 운자를 선택하여 지은 '원시'와 그 운자를 차례대로 사용하여 지은 시인 '차운시'가 있습니다. 중국의 송대 시인들은 이러한 원시와 차운시를 주고받으면서 글로써 대화를 주고받았습니다.  
  
🚩 하지만 여기서 차운시는 상대방이 사용한 운자를 그대로 사용해서 시를 지어야했기 때문에 실제로 시구(문장)의 내용이 어색한 경우가 종종 확인되었습니다. 이러한 차운시의 단점은 고대의 비평가들 역시 지적하기도 했습니다. 
  
🚩 이 프로젝트는 차운시 문장의 부자연스러움을 확인하고자 합니다. MLM 방식으로 학습된 언어모델들을 사용해서 Fill-mask 기법으로 운자를 [MASK]로 처리한 후 Top-7으로 해당 위치의 운자를 언어모델이 예측하게 합니다.
  
🚩 고대중국어를 학습한 모델 4개를 중심으로 실험을 진행하되, 참고로 현대중국어를 학습한 모델 1개를 선정하여 실험하였습니다.

In [2]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 7.7 MB/s 
     |████████████████████████████████| 3.3 MB 48.7 MB/s 
     |████████████████████████████████| 596 kB 72.4 MB/s 
     |████████████████████████████████| 61 kB 682 kB/s 
     |████████████████████████████████| 895 kB 60.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import BertTokenizer, BertForMaskedLM
import pandas as pd
import re

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelWithLMHead, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("ethanyt/guwenbert-large")

model = AutoModelForMaskedLM.from_pretrained("ethanyt/guwenbert-large")

Downloading:   0%|          | 0.00/520 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/91.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

🌹 미니 실험

In [5]:
sequence = f"筑基仿崔巍，鞭石轻险{tokenizer.mask_token}。"
input = tokenizer.encode(sequence, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]
token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 10, dim=1).indices[0].tolist()

top_5_tokens

[6694, 1666, 13410, 2043, 8268, 1856, 426, 1128, 2639, 782]

In [6]:
for token in top_5_tokens:
    print(sequence.replace(tokenizer.mask_token, tokenizer.decode([token])))

筑基仿崔巍，鞭石轻险巇。
筑基仿崔巍，鞭石轻险阻。
筑基仿崔巍，鞭石轻险。
筑基仿崔巍，鞭石轻险峻。
筑基仿崔巍，鞭石轻险。
筑基仿崔巍，鞭石轻险艰。
筑基仿崔巍，鞭石轻险恶。
筑基仿崔巍，鞭石轻险危。
筑基仿崔巍，鞭石轻险隘。
筑基仿崔巍，鞭石轻险夷。


In [7]:
topk_list = []

for token in top_5_tokens:
    topk_list.append(tokenizer.decode([token]))

topk_list

['巇', '阻', '\ue38b', '峻', '\ue48c', '艰', '恶', '危', '隘', '夷']

# 🌹 원시 차운시 실험

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
df = pd.read_csv("/content/drive/MyDrive/mydata6.csv")

In [10]:
# 원시만 모아본다
df_changshi = df[["ow", "ot", "oc"]]
print(df_changshi.shape)
df_changshi.head()

(438, 3)


,ow,ot,oc
0,蘇轍,山胡,山胡拥苍毳，两耳白茸茸。野树啼终日，黔山深几重。啄溪探细石，噪虎上孤峰。被执应多恨，筠笼仅不容。
1,蘇洵,题阎立本画水官,水官骑苍龙，龙行欲上天。手攀时且住，浩若乘风船。不知几何长，足尾犹在渊。下有二从臣，左右乘鱼...
2,蘇轍,怀黾池寄子瞻兄,相携话别郑原上，共道长途怕雪泥。归骑还寻大梁陌，行人已渡古崤西。曾为县吏民知否，旧宿僧房壁共...
3,劉敞,新作石林亭,朝廷入忘返，山林往不还。念无高世姿，聊处可否间。筑基仿崔巍，鞭石轻险艰。群玉相磊落，万峰正孱...
4,蘇轍,辛丑除日寄子瞻,一岁不复居，一日安足惜。人心畏增年，对酒语终夕。夜长书室幽，灯烛明照席。盘飧杂梁楚，羊炙错鱼...


In [11]:
# 비어있는 행을 제거한다
df_changshi.dropna(subset=['oc'], inplace=True)
df_changshi.tail()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ow,ot,oc
432,NaN,NaN,羞看一首回文锦，锦似文君别恨深。头白自吟悲赋客，断肠愁是断弦琴。
433,蘇轍,子瞻与李公麟宣德共画翠石古木老僧谓之憩寂图题其后,东坡自作苍苍石，留取长松待伯时。只有两人嫌未足，更收前世杜陵诗。
435,黃庭堅,摩诘画,丹青王右辖，诗句妙九州。物外常独往，人间无所求。袖手南山雨，辋川桑柘秋。胸中有佳处，泾渭看同流。
436,蘇軾,过岭寄子由,七年来往我何堪，又试曹溪一勺甘。梦里似曾迁海外，醉中不觉到江南。波生濯足鸣空涧，雾绕征衣滴翠...
437,釋了元,偈,赵州当日少谦光，不出三门见赵王。争似金山无量相，大千都是一禅床。


In [ ]:
# 운각을 꺼내서 리스트에 저장
yunjiao_list = []
for sent in pd.DataFrame(df_changshi)["oc"]:
  sentence_list = []
  for index, sentence in enumerate(sent.split('。')[:-1]):
    print(sentence[-1])
    sentence_list.append(sentence[-1])

  yunjiao_list.append(sentence_list)

In [13]:
print(yunjiao_list)

[['茸', '重', '峰', '容'], ['天', '船', '渊', '鼋', '翻', '鬟', '莲', '蛮', '旃', '拳', '鳣', '扳', '川', '烟', '关', '编', '年', '难', '禅', '前'], ['泥', '西', '题', '嘶'], ['还', '间', '艰', '颜', '潺', '攀', '斑', '环', '寰', '关', '山', '閒'], ['惜', '夕', '席', '腊', '昔', '释', '失', '客', '磔', '麦', '拆', '覈', '籍', '炙', '射', '尺', '伯', '碧', '日', '馘', '隔', '掷'], ['囚', '游', '忧', '侯', '流', '浮', '收', '楼', '咻', '幽', '州', '留', '洲', '头', '钩', '刘', '仇', '丘', '驺', '不', '锼', '湫', '偷', '侔', '脩', '旒', '篌', '虬', '眸', '舟', '沟', '秋', '飗', '彪', '辀', '悠', '蝣', '鞦', '呦', '缑', '愁', '搂', '油', '投', '鯈', '喉', '由', '裘', '俦', '酬'], ['升', '灯', '僧', '曾'], ['尖', '纤', '蟾', '帘'], ['啼', '溪', '闺', '梯'], ['心', '深', '簪', '琛'], ['膺', '朋', '登', '应'], ['行', '长', '乡', '藏'], ['来', '才', '腮', '哉'], ['卑', '髭', '师', '差'], ['佐', '货', '大', '卧', '座', '磨', '过', '和'], ['迟', '追', '涯', '时', '肥', '悲', '辞', '衰'], ['蛇', '遮', '何', '哗', '挝', '斜', '跎', '誇'], ['流', '虬', '州', '留', '求', '酬', '悠', '幽', '侯', '头', '浮', '游'], ['归', '非', '飞', '违'], ['鳌', '高', '劳', '袍'], ['无', '姝', '徂', '孤'

In [20]:
df_changshi["yunjiao_list"] = yunjiao_list
df_changshi.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ow,ot,oc,result,yunjiao_list
0,蘇轍,山胡,山胡拥苍毳，两耳白茸茸。野树啼终日，黔山深几重。啄溪探细石，噪虎上孤峰。被执应多恨，筠笼仅不容。,"茸,重,峰,容","[茸, 重, 峰, 容]"
1,蘇洵,题阎立本画水官,水官骑苍龙，龙行欲上天。手攀时且住，浩若乘风船。不知几何长，足尾犹在渊。下有二从臣，左右乘鱼鼋。矍铄相顾视，风举衣袂翻。女子侍君侧，白颊垂双鬟。手执雉尾扇，容如未开莲。从者八九人，非鬼非戎蛮。出水未成列，先登扬旗旃。长刀拥旁牌，白羽注强拳。虽服甲与裳，状貌犹鲸鳣。水兽不得从，仰面以手扳。空虚走雷霆，雨电晦九川。风师黑虎囊，面目昏尘烟。翼从三神人，万里朝天关。我从大觉师，得此诡怪编。画者古阎子，于今三百年。见者谁不爱，予者诚已难。在我犹在子，此理宁非禅。报之以好词，何必画在前。,"天,船,翻,鬟,莲,旃,关,年,前","[天, 船, 渊, 鼋, 翻, 鬟, 莲, 蛮, 旃, 拳, 鳣, 扳, 川, 烟, 关, 编, 年, 难, 禅, 前]"
2,蘇轍,怀黾池寄子瞻兄,相携话别郑原上，共道长途怕雪泥。归骑还寻大梁陌，行人已渡古崤西。曾为县吏民知否，旧宿僧房壁共题。遥想独游佳味少，无言骓马但鸣嘶。,"题,嘶","[泥, 西, 题, 嘶]"
3,劉敞,新作石林亭,朝廷入忘返，山林往不还。念无高世姿，聊处可否间。筑基仿崔巍，鞭石轻险艰。群玉相磊落，万峰正孱颜。种树亦苍苍，激流复潺潺。湡浯歘在眼，昆阆若可攀。自我婴世网，迩来须毛斑。丘壑成弱丧，簿书常自环。及尔灭闻见，旷如远尘寰。岂敢同避世，庶几善闭关。子牟困怀魏，谢傅悲徂山。兹焉可遗老，讵厌终岁閒。,"还,间,颜,潺,关","[还, 间, 艰, 颜, 潺, 攀, 斑, 环, 寰, 关, 山, 閒]"
4,蘇轍,辛丑除日寄子瞻,一岁不复居，一日安足惜。人心畏增年，对酒语终夕。夜长书室幽，灯烛明照席。盘飧杂梁楚，羊炙错鱼腊。庖人馔鸡兔，家味宛如昔。有怀岐山下，展转不能释。念同去闾里，此节三已失。初来寄荆渚，鱼雁贱宜客。楚人重岁时，爆竹鸣磔磔。新春始涉五，田冻未生麦。相携历唐许，花柳渐牙拆。居梁不耐贫，投杞避糠覈。城南庠斋静，终岁守坟籍。酒酸未尝饮，牛美每共炙。谓言从明年，此会可悬射。同为洛中吏，相去不盈尺。浊醪幸分季，新笋可饷伯。巑巑嵩山美，漾漾洛水碧。官闲得相从，春野玩朝日。安知书閤下，群子并遭馘。偶成一朝荣，遂使千里隔。何年相会欢，逢节勿轻掷。,"惜,夕,昔,磔,尺,日","[惜, 夕, 席, 腊, 昔, 释, 失, 客, 磔, 麦, 拆, 覈, 籍, 炙, 射, 尺, 伯, 碧, 日, 馘, 隔, 掷]"


In [14]:
pd.set_option('display.max_colwidth', -1)
mini = df_changshi[2:4]
mini

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,ow,ot,oc
2,蘇轍,怀黾池寄子瞻兄,相携话别郑原上，共道长途怕雪泥。归骑还寻大梁陌，行人已渡古崤西。曾为县吏民知否，旧宿僧房壁共题。遥想独游佳味少，无言骓马但鸣嘶。
3,劉敞,新作石林亭,朝廷入忘返，山林往不还。念无高世姿，聊处可否间。筑基仿崔巍，鞭石轻险艰。群玉相磊落，万峰正孱颜。种树亦苍苍，激流复潺潺。湡浯歘在眼，昆阆若可攀。自我婴世网，迩来须毛斑。丘壑成弱丧，簿书常自环。及尔灭闻见，旷如远尘寰。岂敢同避世，庶几善闭关。子牟困怀魏，谢傅悲徂山。兹焉可遗老，讵厌终岁閒。


In [15]:
df_changshi.head(3)

,ow,ot,oc
0,蘇轍,山胡,山胡拥苍毳，两耳白茸茸。野树啼终日，黔山深几重。啄溪探细石，噪虎上孤峰。被执应多恨，筠笼仅不容。
1,蘇洵,题阎立本画水官,水官骑苍龙，龙行欲上天。手攀时且住，浩若乘风船。不知几何长，足尾犹在渊。下有二从臣，左右乘鱼鼋。矍铄相顾视，风举衣袂翻。女子侍君侧，白颊垂双鬟。手执雉尾扇，容如未开莲。从者八九人，非鬼非戎蛮。出水未成列，先登扬旗旃。长刀拥旁牌，白羽注强拳。虽服甲与裳，状貌犹鲸鳣。水兽不得从，仰面以手扳。空虚走雷霆，雨电晦九川。风师黑虎囊，面目昏尘烟。翼从三神人，万里朝天关。我从大觉师，得此诡怪编。画者古阎子，于今三百年。见者谁不爱，予者诚已难。在我犹在子，此理宁非禅。报之以好词，何必画在前。
2,蘇轍,怀黾池寄子瞻兄,相携话别郑原上，共道长途怕雪泥。归骑还寻大梁陌，行人已渡古崤西。曾为县吏民知否，旧宿僧房壁共题。遥想独游佳味少，无言骓马但鸣嘶。


In [ ]:
def change_word(sentence, tokenizer):
    sentence = re.sub(r'.。', f'{tokenizer.mask_token}。', sentence)
    return sentence

kkk=[]

for sentence in df_changshi["oc"]:
  sequence = change_word(sentence, tokenizer)
  cool_list = []
  for sequence_mask_sentence, raw in zip(sequence.split('。')[:-1], sentence.split('。')[:-1]):
    print(sequence_mask_sentence, raw[-1])

    d_sequence = sequence_mask_sentence+'。'

    tokenizer_input = tokenizer.encode(d_sequence, return_tensors="pt")
    mask_token_index = torch.where(tokenizer_input == tokenizer.mask_token_id)[1]
    token_logits = model(tokenizer_input).logits
    mask_token_logits = token_logits[0, mask_token_index, :]
    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
    topk_list = []

    for token in top_5_tokens:
      topk_list.append(tokenizer.decode([token]))

    print(topk_list)
    if raw[-1] in topk_list:
      cool_list.append(raw[-1])

  result = ' '.join(cool_list)
  kkk.append(result)

In [53]:
print(kkk)

['茸 重 峰 容', '天 船 翻 鬟 莲 旃 关 年 前', '题 嘶', '还 间 颜 潺 关', '惜 夕 昔 磔 尺 日', '囚 游 忧 流 收 楼 州 留 钩 锼 旒 篌 虬 眸 秋 飗 彪 蝣 油 由 裘 酬', '升 曾', '尖 蟾 帘', '啼 溪', '心 深 簪', '膺 朋 应', '行 长 藏', '来 才', '卑 髭 差', '大 卧 过 和', '迟 涯 时 肥 悲 辞', '蛇 何 斜 跎', '酬 悠 侯 头 游', '归 非 飞 违', '袍', '无 徂 孤', '然', '碑 龟 赀 悲', '苔 来 杯', '烟 编 田', '肩 莲 泉 圆 烟 传 观', '', '囚 由 流 秋 抽 鸠 愁', '人 尘 新 轮 巾', '头 楼', '消', '林 心 今 深 簪', '罂 耕', '尊 孙 村 浑', '南 聃', '通 风', '军 群', '可 娜 么', '山', '网 襁', '', '还 山 鬟 弯 寰', '瓢 潮 苗', '潮', '成 名 生', '悠 愁 州', '根 然 蟠 圆 贤', '豪 骚 涛 牢 艘 鳌 劳 旄 醪 蒿', '景 永 省', '胸 同 童 风 穷 东 蓬 翁', '长 昂 冈 茫', '舟', '归 非 依 扉 衣', '阳 光 僵 防 墙 亡 塘 觞', '众 中 梦 瓮', '论 翻 门 拫 源', '宾 绅 臣 邻 人 莘 陈 辰', '欢 宽 漫 团 澜 完 跚 鸾 蟠 抟 酸 弹', '能 丞', '人 邻', '无 呼 娱 纡 孤 蒲 晡 图 蘧 摹', '舟 头 羞 术 蜜 屈 唯 几 名 轻', '船 天', '开 徊', '盘 餐', '翘', '闲 山', '中 穷 功 虫 红 童 通 翁', '川 旋 巅 蜒 禅 钱 煎 便 然 年', '逢 松 容', '雀 愕 薄 从 鸿 中', '堆', '蛇', '应 兴', '凉 妆', '中 蓬', '腾 鹰 冰 憎 缯 登 朋 藤 灯 膺', '巧 丹 寒 斗 酒', '芳 霜', '寺 去', '门 昏 论', '眠', '绝 雪 颊 垂 迟 嬉', '谷 足 盘 湍 席 植 间 山 半 叹', '鳞 新 巾 春', '书 疏 予', '闭 袂 计 继 世', '声 钲 清 耕', '声 耕', 

In [54]:
df_changshi['result'] = kkk
df_changshi.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ow,ot,oc,result,yunjiao_list
0,蘇轍,山胡,山胡拥苍毳，两耳白茸茸。野树啼终日，黔山深几重。啄溪探细石，噪虎上孤峰。被执应多恨，筠笼仅不容。,茸 重 峰 容,"[茸, 重, 峰, 容]"
1,蘇洵,题阎立本画水官,水官骑苍龙，龙行欲上天。手攀时且住，浩若乘风船。不知几何长，足尾犹在渊。下有二从臣，左右乘鱼鼋。矍铄相顾视，风举衣袂翻。女子侍君侧，白颊垂双鬟。手执雉尾扇，容如未开莲。从者八九人，非鬼非戎蛮。出水未成列，先登扬旗旃。长刀拥旁牌，白羽注强拳。虽服甲与裳，状貌犹鲸鳣。水兽不得从，仰面以手扳。空虚走雷霆，雨电晦九川。风师黑虎囊，面目昏尘烟。翼从三神人，万里朝天关。我从大觉师，得此诡怪编。画者古阎子，于今三百年。见者谁不爱，予者诚已难。在我犹在子，此理宁非禅。报之以好词，何必画在前。,天 船 翻 鬟 莲 旃 关 年 前,"[天, 船, 渊, 鼋, 翻, 鬟, 莲, 蛮, 旃, 拳, 鳣, 扳, 川, 烟, 关, 编, 年, 难, 禅, 前]"
2,蘇轍,怀黾池寄子瞻兄,相携话别郑原上，共道长途怕雪泥。归骑还寻大梁陌，行人已渡古崤西。曾为县吏民知否，旧宿僧房壁共题。遥想独游佳味少，无言骓马但鸣嘶。,题 嘶,"[泥, 西, 题, 嘶]"
3,劉敞,新作石林亭,朝廷入忘返，山林往不还。念无高世姿，聊处可否间。筑基仿崔巍，鞭石轻险艰。群玉相磊落，万峰正孱颜。种树亦苍苍，激流复潺潺。湡浯歘在眼，昆阆若可攀。自我婴世网，迩来须毛斑。丘壑成弱丧，簿书常自环。及尔灭闻见，旷如远尘寰。岂敢同避世，庶几善闭关。子牟困怀魏，谢傅悲徂山。兹焉可遗老，讵厌终岁閒。,还 间 颜 潺 关,"[还, 间, 艰, 颜, 潺, 攀, 斑, 环, 寰, 关, 山, 閒]"
4,蘇轍,辛丑除日寄子瞻,一岁不复居，一日安足惜。人心畏增年，对酒语终夕。夜长书室幽，灯烛明照席。盘飧杂梁楚，羊炙错鱼腊。庖人馔鸡兔，家味宛如昔。有怀岐山下，展转不能释。念同去闾里，此节三已失。初来寄荆渚，鱼雁贱宜客。楚人重岁时，爆竹鸣磔磔。新春始涉五，田冻未生麦。相携历唐许，花柳渐牙拆。居梁不耐贫，投杞避糠覈。城南庠斋静，终岁守坟籍。酒酸未尝饮，牛美每共炙。谓言从明年，此会可悬射。同为洛中吏，相去不盈尺。浊醪幸分季，新笋可饷伯。巑巑嵩山美，漾漾洛水碧。官闲得相从，春野玩朝日。安知书閤下，群子并遭馘。偶成一朝荣，遂使千里隔。何年相会欢，逢节勿轻掷。,惜 夕 昔 磔 尺 日,"[惜, 夕, 席, 腊, 昔, 释, 失, 客, 磔, 麦, 拆, 覈, 籍, 炙, 射, 尺, 伯, 碧, 日, 馘, 隔, 掷]"


In [61]:
all_yunjiao = df_changshi["yunjiao_list"].str.len()
df_changshi['all_yunjiao']=all_yunjiao

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
# 차운시만 모아본다

test = df[["rc1", "rc2","rc3","rc4","rc5", "rc6","rc7","rc8"]]
print(df_ciyun.shape)
test_rc1 = test[["rc1"]].dropna(axis=0)
test_rc2 = test[["rc2"]].dropna(axis=0)
test_rc3 = test[["rc3"]].dropna(axis=0)
test_rc4 = test[["rc4"]].dropna(axis=0)
test_rc5 = test[["rc5"]].dropna(axis=0)
test_rc6 = test[["rc6"]].dropna(axis=0)
test_rc7 = test[["rc7"]].dropna(axis=0)
test_rc8 = test[["rc8"]].dropna(axis=0)

(438, 8)


In [35]:
test_rc1.columns = ['contents']
test_rc2.columns = ['contents']
test_rc3.columns = ['contents']
test_rc4.columns = ['contents']
test_rc5.columns = ['contents']
test_rc6.columns = ['contents']
test_rc7.columns = ['contents']
test_rc8.columns = ['contents']

In [38]:
test = pd.concat([test_rc1, test_rc2,test_rc3, test_rc4,
                   test_rc5, test_rc6, test_rc7, test_rc8], ignore_index = True)
test.columns = ['poem']
print(test.shape)
test.head()

(596, 1)


,poem
0,终日锁筠笼，回头惜翠茸。谁知声㗲㗲，亦自意重重。夜宿烟生浦，朝鸣日上峰。故巢何足恋，鹰隼岂能容。
1,高人岂学画，用笔乃其天。譬如善游人，一一能操船。阎子本缝掖，畴昔慕云渊。丹青偶为戏，染指初尝鼋。爱之不自已，笔势如风翻。传闻贞观中，左衽解椎鬟。南夷羞白雉，佛国贡青莲。诏令拟王会，别殿写戎蛮。熊冠金络额，豹袖拥幡旃。传入应门内，俯伏脱剑弮。天姿俨龙凤，杂沓朝鹏鳣。神功与绝迹，后世两莫扳。自从李氏亡，群盗窃山川。长安三日火，至宝随飞烟。尚有脱身者，漂流东出关。三官岂容独，得此今已偏。吁嗟至神物，会合当有年。京城诸权贵，欲取百计难。赠以玉如意，岂能动高禅。惟应一篇诗，皎若画在前。
2,人生到处知何似，应似飞鸿踏雪泥。泥上偶然留指爪，鸿飞那复计东西。老僧已死成新塔，坏壁无由见旧题。往日崎岖还记否，路长人困蹇驴嘶。
3,都城日荒废，往事不可还。惟馀古苑石，漂散尚人间。公来始购蓄，不惮道里艰。忽从尘埃中，来对冰雪颜。瘦骨拔凛凛，苍根漱潺潺。唐人惟奇章，好石古莫攀。尽令属牛氏，刻凿纷斑斑。嗟此本何常，聚散实循环。人失亦人得，要不出区寰。君看刘李末，不能保河关。况此百株石，鸿毛于泰山。但当对石饮，万事付等闲。
4,薄宦驱我西，远别不容惜。方愁后会远，未暇忧岁夕。强欢虽有酒，冷酌不成席。秦烹惟羊羹，陇馔有熊腊。念为儿童岁，屈指已成昔。往事今何追，忽若箭已释。感时嗟事变，所得不偿失。府卒来驱傩，矍铄惊远客。愁来岂有魔，烦汝为攘磔。寒梅与冻杏，嫩萼初似麦。攀条为惆怅，玉蕊何时折。不忧春艳晚，行见弃夏覈。人生行乐耳，安用声名籍。胡为独多感，不见膏自炙。诗来苦相宽，子意远可射。依依见其面，疑子在咫尺。兄今虽小官，幸忝佐方伯。北池近所凿，中有汧水碧。临池饮美酒，尚可消永日。但恐诗力弱，斗健未免馘。诗成十日到，谁谓千里隔。一月寄一篇，忧愁何足掷。


In [ ]:
ccc=[]

for sentence in test["poem"]:
  sequence = change_word(sentence, tokenizer)
  cool_list = []
  for sequence_mask_sentence, raw in zip(sequence.split('。')[:-1], sentence.split('。')[:-1]):
    print(sequence_mask_sentence, raw[-1])

    d_sequence = sequence_mask_sentence+'。'

    tokenizer_input = tokenizer.encode(d_sequence, return_tensors="pt")
    mask_token_index = torch.where(tokenizer_input == tokenizer.mask_token_id)[1]
    token_logits = model(tokenizer_input).logits
    mask_token_logits = token_logits[0, mask_token_index, :]
    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
    topk_list = []

    for token in top_5_tokens:
      topk_list.append(tokenizer.decode([token]))

    print(topk_list)
    if raw[-1] in topk_list:
      cool_list.append(raw[-1])

  result = ' '.join(cool_list)
  ccc.append(result)

In [43]:
test['result'] = ccc
test.head()

,poem,result
0,终日锁筠笼，回头惜翠茸。谁知声㗲㗲，亦自意重重。夜宿烟生浦，朝鸣日上峰。故巢何足恋，鹰隼岂能容。,重 容
1,高人岂学画，用笔乃其天。譬如善游人，一一能操船。阎子本缝掖，畴昔慕云渊。丹青偶为戏，染指初尝鼋。爱之不自已，笔势如风翻。传闻贞观中，左衽解椎鬟。南夷羞白雉，佛国贡青莲。诏令拟王会，别殿写戎蛮。熊冠金络额，豹袖拥幡旃。传入应门内，俯伏脱剑弮。天姿俨龙凤，杂沓朝鹏鳣。神功与绝迹，后世两莫扳。自从李氏亡，群盗窃山川。长安三日火，至宝随飞烟。尚有脱身者，漂流东出关。三官岂容独，得此今已偏。吁嗟至神物，会合当有年。京城诸权贵，欲取百计难。赠以玉如意，岂能动高禅。惟应一篇诗，皎若画在前。,烟 关 年 前
2,人生到处知何似，应似飞鸿踏雪泥。泥上偶然留指爪，鸿飞那复计东西。老僧已死成新塔，坏壁无由见旧题。往日崎岖还记否，路长人困蹇驴嘶。,泥 西 题
3,都城日荒废，往事不可还。惟馀古苑石，漂散尚人间。公来始购蓄，不惮道里艰。忽从尘埃中，来对冰雪颜。瘦骨拔凛凛，苍根漱潺潺。唐人惟奇章，好石古莫攀。尽令属牛氏，刻凿纷斑斑。嗟此本何常，聚散实循环。人失亦人得，要不出区寰。君看刘李末，不能保河关。况此百株石，鸿毛于泰山。但当对石饮，万事付等闲。,间 颜 潺 斑 环 山 闲
4,薄宦驱我西，远别不容惜。方愁后会远，未暇忧岁夕。强欢虽有酒，冷酌不成席。秦烹惟羊羹，陇馔有熊腊。念为儿童岁，屈指已成昔。往事今何追，忽若箭已释。感时嗟事变，所得不偿失。府卒来驱傩，矍铄惊远客。愁来岂有魔，烦汝为攘磔。寒梅与冻杏，嫩萼初似麦。攀条为惆怅，玉蕊何时折。不忧春艳晚，行见弃夏覈。人生行乐耳，安用声名籍。胡为独多感，不见膏自炙。诗来苦相宽，子意远可射。依依见其面，疑子在咫尺。兄今虽小官，幸忝佐方伯。北池近所凿，中有汧水碧。临池饮美酒，尚可消永日。但恐诗力弱，斗健未免馘。诗成十日到，谁谓千里隔。一月寄一篇，忧愁何足掷。,腊 失 尺 伯 日 隔


In [ ]:
yunjiao_list2 = []
for sent in pd.DataFrame(test)["poem"]:
  sentence_list = []
  for index, sentence in enumerate(sent.split('。')[:-1]):
    print(sentence[-1])
    sentence_list.append(sentence[-1])

  yunjiao_list2.append(sentence_list)

In [51]:
test["yunjiao_list"] = yunjiao_list2
test.head(10)

,poem,result,yunjiao_list
0,终日锁筠笼，回头惜翠茸。谁知声㗲㗲，亦自意重重。夜宿烟生浦，朝鸣日上峰。故巢何足恋，鹰隼岂能容。,重 容,"[茸, 重, 峰, 容]"
1,高人岂学画，用笔乃其天。譬如善游人，一一能操船。阎子本缝掖，畴昔慕云渊。丹青偶为戏，染指初尝鼋。爱之不自已，笔势如风翻。传闻贞观中，左衽解椎鬟。南夷羞白雉，佛国贡青莲。诏令拟王会，别殿写戎蛮。熊冠金络额，豹袖拥幡旃。传入应门内，俯伏脱剑弮。天姿俨龙凤，杂沓朝鹏鳣。神功与绝迹，后世两莫扳。自从李氏亡，群盗窃山川。长安三日火，至宝随飞烟。尚有脱身者，漂流东出关。三官岂容独，得此今已偏。吁嗟至神物，会合当有年。京城诸权贵，欲取百计难。赠以玉如意，岂能动高禅。惟应一篇诗，皎若画在前。,烟 关 年 前,"[天, 船, 渊, 鼋, 翻, 鬟, 莲, 蛮, 旃, 弮, 鳣, 扳, 川, 烟, 关, 偏, 年, 难, 禅, 前]"
2,人生到处知何似，应似飞鸿踏雪泥。泥上偶然留指爪，鸿飞那复计东西。老僧已死成新塔，坏壁无由见旧题。往日崎岖还记否，路长人困蹇驴嘶。,泥 西 题,"[泥, 西, 题, 嘶]"
3,都城日荒废，往事不可还。惟馀古苑石，漂散尚人间。公来始购蓄，不惮道里艰。忽从尘埃中，来对冰雪颜。瘦骨拔凛凛，苍根漱潺潺。唐人惟奇章，好石古莫攀。尽令属牛氏，刻凿纷斑斑。嗟此本何常，聚散实循环。人失亦人得，要不出区寰。君看刘李末，不能保河关。况此百株石，鸿毛于泰山。但当对石饮，万事付等闲。,间 颜 潺 斑 环 山 闲,"[还, 间, 艰, 颜, 潺, 攀, 斑, 环, 寰, 关, 山, 闲]"
4,薄宦驱我西，远别不容惜。方愁后会远，未暇忧岁夕。强欢虽有酒，冷酌不成席。秦烹惟羊羹，陇馔有熊腊。念为儿童岁，屈指已成昔。往事今何追，忽若箭已释。感时嗟事变，所得不偿失。府卒来驱傩，矍铄惊远客。愁来岂有魔，烦汝为攘磔。寒梅与冻杏，嫩萼初似麦。攀条为惆怅，玉蕊何时折。不忧春艳晚，行见弃夏覈。人生行乐耳，安用声名籍。胡为独多感，不见膏自炙。诗来苦相宽，子意远可射。依依见其面，疑子在咫尺。兄今虽小官，幸忝佐方伯。北池近所凿，中有汧水碧。临池饮美酒，尚可消永日。但恐诗力弱，斗健未免馘。诗成十日到，谁谓千里隔。一月寄一篇，忧愁何足掷。,腊 失 尺 伯 日 隔,"[惜, 夕, 席, 腊, 昔, 释, 失, 客, 磔, 麦, 折, 覈, 籍, 炙, 射, 尺, 伯, 碧, 日, 馘, 隔, 掷]"
5,山川足清旷，阛阓巧拘囚。安得孅阿御，同为穆满游。遥知因涣汗，远出散幽忧。原隰繁分绣，村墟尽小侯。春深秦树绿，野阔渭河流。四顾神萧瑟，前探意涨浮。胜观殊未已，往足讵能收。下坂如浮舸，登崖剧上楼。强行腰伛偻，困坐气嘘咻。鸟语林峦静，花明涧谷幽。濯溪惊野老，伐路骇他州。中散探深去，文渊到处留。听琴峰下寺，弄石水中洲。溪冷泉冰脚，山高雾绕头。石潭清照骨，瀑水溅成钩。仙庙鸣钟磬，神官秉钺刘。养生闻帝女，服气绝彭雠。故宅犹传尹，先师不喜丘。居人那识道，过客谩停驺。岩谷诚深绝，神仙信有不。云居无几杖，霞佩弃镌锼。豹隐连山雾，龙潜百尺湫。门开谁与叩，桃熟浪传偷。绀发清无比，方瞳凛不侔。会须林下见，乞取寿年修。拔去和鸡犬，相随若旆旒。乘风遗騕袅，长啸贱笙篌。从骑衣皆羽，前驱鬣尽虬。安能牵两足，暂得快双眸。自昔辞乡树，南行上楚舟。万江穷地脉，三峡束天沟。云暗酆都晚，波吹木枥秋。寻溪缘窈窕，入洞听飕飗。空寺收黄栗，荒祠画伏彪。登临虽永日，行迈肯停辀。蓄缩今何事，攀跻昔已悠。魏京饶士女，春服聚蜉蝣。雷动车争陌，花摇树系鞦。游人纷荡漾，野鸟自嘤呦。平日曾经洛，闲居愿卜缑。空言真比梦，久渴渐成愁。早退尝相约，辞嚣痛自搂。爱山心劫劫，从宦兴油油。海宇都无碍，山林尽可投。愿为云上鹄，莫作盎中鯈。适性行随足，谋生富给喉。今游虽不与，后会岂无由。昼出同穿履，宵眠共覆裘。弟兄真欲尔，朋好定谁俦。试写长篇调，何人肯见酬。,囚 忧 流 幽 头 偷 眸 秋 飗 蝣 油 裘,"[囚, 游, 忧, 侯, 流, 浮, 收, 楼, 咻, 幽, 州, 留, 洲, 头, 钩, 刘, 雠, 丘, 驺, 不, 锼, 湫, 偷, 侔, 修, 旒, 篌, 虬, 眸, 舟, 沟, 秋, 飗, 彪, 辀, 悠, 蝣, 鞦, 呦, 缑, 愁, 搂, 油, 投, 鯈, 喉, 由, 裘, 俦, 酬]"
6,山下晨光晚，林梢露滴升。峰头斜见月，野市早明灯。树暗犹藏鹊，堂开已馔僧。据鞍应梦我，联骑昔尝曾。,灯,"[升, 灯, 僧, 曾]"
7,谢公游意未能厌，踏尽登山屐齿尖。古殿神仙深杳杳，香炉烟翠起纤纤。岩花寂历飘琼片，庭桧萧疏漏玉蟾。帝子莫归人不见，微风细雨自开帘。,尖 纤 帘,"[尖, 纤, 蟾, 帘]"
8,潭边沙水不成泥，潭上孤禽挂崄啼。缭绕飞桥能试客，蒙茸翠蔓巧藏溪。云为绛帐马融室，石作屏风玉女闺。仙果知君今未足，临潭脚战怕长梯。,,"[啼, 溪, 闺, 梯]"
9,秋气萧骚仍见雪，客愁缭绕动萦心。幽吟北户窗声细，归梦函关马迹深。疏树飞花轻蔌蔌，衰荷留柄乱簪簪。遥闻诗酒皆推胜，社客何人近纳琛。,心 深 簪,"[心, 深, 簪, 琛]"


In [64]:
all_yunjiao = test["yunjiao_list"].str.len()
test['all_yunjiao']=all_yunjiao

In [68]:
# 실제 운자 중에서 예측한 개수가 몇 개인지 계산

def no_space(sentence):
  sentence = len(sentence.replace(" ", ""))
  return sentence

test["processed_list"] = test["result"].apply(no_space)
df_changshi["processed_list"] = df_changshi["result"].apply(no_space)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [81]:
df_changshi["total"] = df_changshi["processed_list"]/df_changshi["all_yunjiao"]
test["total"] = test["processed_list"]/test["all_yunjiao"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [83]:
df_changshi["total"].mean() # 원시와 비교하기 위해 참고로 통계 처리

0.6546322861973356

In [82]:
test["total"].mean()

0.5672753917946508

In [84]:
df_changshi.to_csv("changshi_yunjiao.csv", encoding='utf-8', index=False)
test.to_csv("ciyun_yunjiao.csv", encoding='utf-8', index=False)

In [85]:
changshi_yunjiao = pd.read_csv("/content/changshi_yunjiao.csv")
ciyun_yunjiao = pd.read_csv("/content/ciyun_yunjiao.csv")

In [96]:
changshi_sorted = changshi_yunjiao.sort_values(by = ["all_yunjiao"], ascending = False)
changshi_sorted.head(2)

,ow,ot,oc,result,yunjiao_list,all_yunjiao,processed_list,total
5,蘇軾,壬寅二月，有诏令郡吏分往属县减决囚禁。自十三日受命出府，至宝鸡虢郿盩厔四县。既毕事，因朝谒太平宫，而宿于南溪溪堂，遂并南山而西,远人罹水旱，王命释俘囚。分县传明诏，寻山得胜游。萧条初出郭，旷荡实消忧。薄暮来孤镇，登临忆武侯。峥嵘依绝壁，苍茫瞰奔流。半夜人呼急，横空火气浮。天遥殊不辨，风急已难收。晓入陈仓县，犹馀卖酒楼。烟煤已狼藉，吏卒尚呀咻。鸡岭云霞古，龙宫殿宇幽。南山连大散，归路走吾州。欲往安能遂，将还为少留。回趋西虢道，却渡小河洲。闻道磻溪石，犹存渭水头。苍崖虽有迹，大钓本无钩。东去过郿坞，孤城象汉刘。谁言董公健，竟复伍孚仇。白刃俄生肘，黄金谩似丘。平生闻太白，一见驻行驺。鼓角谁能试，风雷果致不。岩崖已奇绝，冰雪更雕锼。春旱忧无麦，山灵喜有湫。蛟龙懒方睡，瓶罐小容偷。二曲林泉胜，三川气象侔。近山麰麦早，临水竹篁脩。先帝膺符命，行宫画冕旒。侍臣簪武弁，女乐抱箜篌。秘殿开金锁，神人控玉虬。黑衣横巨剑，被发凛双眸。邂逅逢佳士，相将弄彩舟。投篙披绿荇，濯足乱清沟。晚宿南溪上，森如水国秋。绕湖栽翠密，终夜响飕飗。冒晓穷幽邃，操戈畏炳彪。尹生犹有宅，老氏旧停辀。问道遗踪在，登仙往事悠。御风归汗漫，阅世似蜉蝣。羽客知人意，瑶琴系马鞦。不辞山寺远，来作鹿鸣呦。帝子传闻李，岩堂髣像缑。轻风帏幔卷，落日髻鬟愁。入谷惊蒙密，登坡费挽搂。乱峰搀似槊，一水澹如油。中使何年到，金龙自古投。千重横翠石，百丈见游鯈。最爱泉鸣洞，初尝雪入喉。满瓶虽可致，洗耳叹无由。忽忆寻蟆培，方冬脱鹿裘。山川良甚似，水石亦堪俦。惟有泉傍饮，无人自献酬。,囚 游 忧 流 收 楼 州 留 钩 锼 旒 篌 虬 眸 秋 飗 彪 蝣 油 由 裘 酬,"['囚', '游', '忧', '侯', '流', '浮', '收', '楼', '咻', '幽', '州', '留', '洲', '头', '钩', '刘', '仇', '丘', '驺', '不', '锼', '湫', '偷', '侔', '脩', '旒', '篌', '虬', '眸', '舟', '沟', '秋', '飗', '彪', '辀', '悠', '蝣', '鞦', '呦', '缑', '愁', '搂', '油', '投', '鯈', '喉', '由', '裘', '俦', '酬']",50,22,0.440
57,蘇軾,甘露寺,江山岂不好，独游情易阑。但有相携人，何必素所欢。我欲访甘露，当途无闲官。二子旧不识，欣然肯联鞍。古郡山为城，层梯转朱栏。楼台断崖上，地窄天水宽。一览吞数州，山长江漫漫。却望大明寺，惟见烟中竿。狠石卧庭下，穹窿如伏羱。缅怀卧龙公，挟策事雕钻。一谈收猘子，再说走老瞒。名高有馀想，事往无留观。萧公古铁镬，相对空团团。陂陀受百斛，积雨生微澜。泗水逸周鼎，渭城辞汉盘。山川失故态，怪此能独完。僧繇六化人，霓衣挂冰纨。隐见十二叠，观者疑夸谩。破板陆生画，青猊戏盘跚。上有二天人，挥手如翔鸾。笔墨虽欲尽，典型垂不刊。赫赫赞皇公，英姿凛以寒。古柏手亲种，挺然谁敢干。枝撑云峰裂，根入石窟蟠。薙草得断碑，斩崖出金棺。瘗藏岂不牢，见伏理可叹。四雄皆龙虎，遗迹俨未刓。方其盛壮时，争夺肯少安。废兴属造物，迁逝谁控抟。况彼妄庸子，而欲事所难。古今共一轨，后世徒辛酸。聊兴广武叹，不待雍门弹。,欢 宽 漫 团 澜 完 跚 鸾 蟠 抟 酸 弹,"['阑', '欢', '官', '鞍', '栏', '宽', '漫', '竿', '羱', '钻', '瞒', '观', '团', '澜', '盘', '完', '纨', '谩', '跚', '鸾', '刊', '寒', '干', '蟠', '棺', '叹', '刓', '安', '抟', '难', '酸', '弹']",32,12,0.375


In [97]:
ciyun_sorted = ciyun_yunjiao.sort_values(by = ["all_yunjiao"], ascending = False)
ciyun_sorted.head(2)

,poem,result,yunjiao_list,all_yunjiao,processed_list,total
5,山川足清旷，阛阓巧拘囚。安得孅阿御，同为穆满游。遥知因涣汗，远出散幽忧。原隰繁分绣，村墟尽小侯。春深秦树绿，野阔渭河流。四顾神萧瑟，前探意涨浮。胜观殊未已，往足讵能收。下坂如浮舸，登崖剧上楼。强行腰伛偻，困坐气嘘咻。鸟语林峦静，花明涧谷幽。濯溪惊野老，伐路骇他州。中散探深去，文渊到处留。听琴峰下寺，弄石水中洲。溪冷泉冰脚，山高雾绕头。石潭清照骨，瀑水溅成钩。仙庙鸣钟磬，神官秉钺刘。养生闻帝女，服气绝彭雠。故宅犹传尹，先师不喜丘。居人那识道，过客谩停驺。岩谷诚深绝，神仙信有不。云居无几杖，霞佩弃镌锼。豹隐连山雾，龙潜百尺湫。门开谁与叩，桃熟浪传偷。绀发清无比，方瞳凛不侔。会须林下见，乞取寿年修。拔去和鸡犬，相随若旆旒。乘风遗騕袅，长啸贱笙篌。从骑衣皆羽，前驱鬣尽虬。安能牵两足，暂得快双眸。自昔辞乡树，南行上楚舟。万江穷地脉，三峡束天沟。云暗酆都晚，波吹木枥秋。寻溪缘窈窕，入洞听飕飗。空寺收黄栗，荒祠画伏彪。登临虽永日，行迈肯停辀。蓄缩今何事，攀跻昔已悠。魏京饶士女，春服聚蜉蝣。雷动车争陌，花摇树系鞦。游人纷荡漾，野鸟自嘤呦。平日曾经洛，闲居愿卜缑。空言真比梦，久渴渐成愁。早退尝相约，辞嚣痛自搂。爱山心劫劫，从宦兴油油。海宇都无碍，山林尽可投。愿为云上鹄，莫作盎中鯈。适性行随足，谋生富给喉。今游虽不与，后会岂无由。昼出同穿履，宵眠共覆裘。弟兄真欲尔，朋好定谁俦。试写长篇调，何人肯见酬。,囚 忧 流 幽 头 偷 眸 秋 飗 蝣 油 裘,"['囚', '游', '忧', '侯', '流', '浮', '收', '楼', '咻', '幽', '州', '留', '洲', '头', '钩', '刘', '雠', '丘', '驺', '不', '锼', '湫', '偷', '侔', '修', '旒', '篌', '虬', '眸', '舟', '沟', '秋', '飗', '彪', '辀', '悠', '蝣', '鞦', '呦', '缑', '愁', '搂', '油', '投', '鯈', '喉', '由', '裘', '俦', '酬']",50,12,0.240000
57,去国日已远，涉江岁将阑。东南富山水，跬步留清欢。迁延废行迈，忽忘身在官。清晨涉甘露，乘高弃征鞍。超然脱阛阓，穿云抚朱栏。下视万物微，惟觉沧海宽。潮来声汹汹，望极空漫漫。一一渡海舶，冉冉移樯竿。水怪时出没，群嬉类豭羱。幽阴自生火，青荧复谁钻。石头古天险，凭恃分权瞒。疑城曜远目，来骑惊新观。聚散定王业，成毁犹月团。金山百围石，岌岌随涛澜。犹疑汉宫廷，屹立承露盘。狂波恣吞噬，万古嗟独完。凝眸厌滉漾，绕屋行盘跚。此寺历今古，遗迹皆龙鸾。孔明所坐石，牂䍽非人刊。经霜众草短，积雨青苔寒。萧翁嗜佛法，大福将力干。坡陀故镬在，甲错苍龙蟠。卫公秉节制，佛骨埋金棺。长松看百尺，画像留三叹。新诗语何丽，传读纸遂刓。嗟我本渔钓，江湖心所安。方为笼中闭，仰羡天际抟。游观惜不与，赋咏嗟独难。俸禄藉升斗，齑盐嗜咸酸。何时扁舟去，不俟官长弹。,栏 漫 竿 盘 跚 蟠 棺 叹 安 难 酸,"['阑', '欢', '官', '鞍', '栏', '宽', '漫', '竿', '羱', '钻', '瞒', '观', '团', '澜', '盘', '完', '跚', '鸾', '刊', '寒', '干', '蟠', '棺', '叹', '刓', '安', '抟', '难', '酸', '弹']",30,11,0.366667
